In [ ]:
pip install -U langchain langchain-core langchain-community langchain-text-splitters gpt4all pymupdf gradio

In [ ]:
import pymupdf as fitz
import gradio as gr
import textwrap
from langchain_community.llms import GPT4All
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

# -----------------------------
# Initialize GPT model
# -----------------------------
def load_gpt4all_model(model_path=r"C:\Users\Omaid\AppData\Local\nomic.ai\GPT4All\Meta-Llama-3-8B-Instruct.Q4_0.gguf"):
    return GPT4All(model=model_path, verbose=False)

llm = load_gpt4all_model()

# -----------------------------
# Prompt Template
# -----------------------------
prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text:\n\n{text}\n\nSummary:",
)

# -----------------------------
# Create chain
# -----------------------------
chain = RunnableSequence(prompt | llm)

# -----------------------------
# PDF Extraction
# -----------------------------
def extractPDF(pdf, maxPages=5):
    textContent = ""
    try:
        pdf_path = pdf.name if hasattr(pdf, "name") else pdf
        doc = fitz.open(pdf_path)
        for i in range(min(maxPages, len(doc))):
            textContent += doc[i].get_text("text")
    except Exception as e:
        return f"Error reading PDF: {e}"
    return textContent.strip() or "No extractable text found in PDF."

# -----------------------------
# Summarize text
# -----------------------------
def summarize_text(fullText, chunkSize=1000):
    if len(fullText) <= chunkSize:
        return chain.invoke({"text": fullText})

    chunks = textwrap.wrap(fullText, chunkSize)
    summaries = []
    for chunk in chunks:
        try:
            summaries.append(chain.invoke({"text": chunk}))
        except Exception as e:
            summaries.append(f"[Error summarizing chunk: {e}]")
    return " ".join(summaries)

# -----------------------------
# Gradio
# -----------------------------
def generateSummary(input_text, pdf):
    if pdf is not None:
        textContent = extractPDF(pdf)
        if textContent:
            return summarize_text(textContent)
        else:
            return "No text could be extracted from the PDF."
    elif input_text:
        return summarize_text(input_text)
    else:
        return "Please provide input text or upload a PDF file."

# -----------------------------
# Gradio Interface
# -----------------------------
interface = gr.Interface(
    fn=generateSummary,
    inputs=[
        gr.Textbox(lines=10, label="Input Text (leave blank if uploading PDF)"),
        gr.File(label="Upload PDF"),
    ],
    outputs=gr.Textbox(label="Summary", lines=15),
    title="PDF & Text Summarizer",
    description="Summarize text from an uploaded PDF or manually entered text",
)

# -----------------------------
# Launch App
# -----------------------------
interface.launch(share=False, inbrowser=True)